In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from custom_py_modules import *
from scipy.optimize import differential_evolution
import json

In [ ]:
# Load three years of data

# 5 years in dataframe
df_load = pd.concat([pd.read_csv('./test_simulink/E-load-2013.csv'), 
                     pd.read_csv('./test_simulink/E-load-2014.csv'), 
                     pd.read_csv('./test_simulink/E-load-2015.csv'),
                     pd.read_csv('./test_simulink/E-load-2017.csv'),
                     pd.read_csv('./test_simulink/E-load-2018.csv')
                     ])
df_weather = pd.concat([pd.read_csv('./test_simulink/E-2013.csv'), 
                        pd.read_csv('./test_simulink/E-2014.csv'), 
                        pd.read_csv('./test_simulink/E-2015.csv'),
                        pd.read_csv('./test_simulink/E-2017.csv'),
                        pd.read_csv('./test_simulink/E-2018.csv')
                        ])

# Combine irradiance
df_weather['irradiance_total'] = df_weather['irradiance_direct'] + df_weather['irradiance_diffuse']

# Change the time column names
df_weather['timestamp'] = df_weather['local_time_no_dst']
df_weather = df_weather.drop(columns=['local_time_no_dst'])

df_load['timestamp'] = df_load['DD/MM/YYYY HH:MM']
df_load = df_load.drop(columns=['DD/MM/YYYY HH:MM'])

# Convert to datetime
df_load['timestamp'] = pd.to_datetime(df_load['timestamp'], format='%d/%m/%Y %H:%M')
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'], format='%Y-%m-%d %H:%M:%S')
# Set timestamp as index
df_load = df_load.set_index('timestamp')
df_weather = df_weather.set_index('timestamp')

# Add kW column
df_load['Load (kW)'] = df_load['Load (Watt)'] / 1000
df_load_hourly = df_load.resample('H').mean()

df_load_hourly_2013 = df_load_hourly.loc['2013-01-01':'2013-12-31']
df_load_hourly_2014 = df_load_hourly.loc['2014-01-01':'2014-12-31']
df_load_hourly_2015 = df_load_hourly.loc['2015-01-01':'2015-12-31']
df_load_hourly_2017 = df_load_hourly.loc['2017-01-01':'2017-12-31']
df_load_hourly_2018 = df_load_hourly.loc['2018-01-01':'2018-12-31']

df_weather_hourly_2013 = df_weather.loc['2013-01-01':'2013-12-31']
df_weather_hourly_2014 = df_weather.loc['2014-01-01':'2014-12-31']
df_weather_hourly_2015 = df_weather.loc['2015-01-01':'2015-12-31']
df_weather_hourly_2017 = df_weather.loc['2017-01-01':'2017-12-31']
df_weather_hourly_2018 = df_weather.loc['2018-01-01':'2018-12-31']

In [ ]:
# Define wind turbines
gazelle_turbine = WindTurbine(blade_radius=5, # [m]
                              performance_coefficient=0.4, # [-]
                              air_density=1.225, # [kg/m^3]
                              nominal_power=20, # [kW] 
                              nominal_wind_speed=13, # [m/s]
                              cut_out_speed=20, # [m/s] 
                              cut_in_speed=4, # [m/s] 
                              price=60000, # EUR from alibaba
                              lifetime=22.5, # years 
                              height=18) # [m]

aircon_hawt_turbine = WindTurbine(blade_radius=3.5, # [m]
                              performance_coefficient=0.35, # [-]
                              air_density=1.225, # [kg/m^3]
                              nominal_power=10, # [kW] 
                              nominal_wind_speed=11, # [m/s]
                              cut_out_speed=32, # [m/s] 
                              cut_in_speed=2.5, # [m/s] 
                              price=35000, # EUR 
                              lifetime=25, # years 
                              height=18) # [m]

travere_turbine = WindTurbine(blade_radius=1.6, # [m]
                              performance_coefficient=0.35, # [-]
                                air_density=1.225, # [kg/m^3]
                                nominal_power=1.6, # [kW]
                                nominal_wind_speed=10, # [m/s]
                                cut_out_speed=60, # [m/s]
                                cut_in_speed=2.5 , # [m/s]
                                price=7000, # EUR
                                lifetime=25, # years
                                height=12) # [m]

wf_builder = WindFarmBuilder()
wind_speed_array = df_weather['wind_speed'].to_numpy()
print(wind_speed_array)
print("20kW annual energy", gazelle_turbine.get_annual_energy_production(wind_speed_array, years=5))
print("10kW annual energy", aircon_hawt_turbine.get_annual_energy_production(wind_speed_array, years=5))
print("1.6kW annual energy", travere_turbine.get_annual_energy_production(wind_speed_array, years=5))

In [ ]:
# Print wind turbine capacity factors

# Gazelle simple capacity factor
print("Gazelle simple capacity factor", gazelle_turbine.get_capacity_factor(wind_speed_array, years=5, simple=True))
# Gazelle physical capacity factor
print("Gazelle physical capacity factor", gazelle_turbine.get_capacity_factor(wind_speed_array, years=5, simple=False))
# Aircon simple capacity factor
print("Aircon simple capacity factor", aircon_hawt_turbine.get_capacity_factor(wind_speed_array, years=5, simple=True))
# Aircon physical capacity factor
print("Aircon physical capacity factor", aircon_hawt_turbine.get_capacity_factor(wind_speed_array, years=5, simple=False))
# Travere simple capacity factor
print("Travere simple capacity factor", travere_turbine.get_capacity_factor(wind_speed_array, years=5, simple=True))
# Travere physical capacity factor
print("Travere physical capacity factor", travere_turbine.get_capacity_factor(wind_speed_array, years=5, simple=False))

In [ ]:
# In this cell, define the energy system

# Wind farm
sample_wind_farm = wf_builder.build_wind_farm([gazelle_turbine, aircon_hawt_turbine, travere_turbine], [7, 0, 0])

# Solar farm
sample_solar_farm = SolarFarm(area=0, # [m^2] 
                                pricepersqm= 325 # in E
                                )

# MicroGrid
sample_microgrid = MicroGrid(windfarm = sample_wind_farm, 
                                solarfarm= sample_solar_farm,
                                storage_capacity_kwh= 13.5*80, # [kWh]
                                storage_efficiency=0.8, # [-] 
                                storage_price_per_kw= 1000 # [EUR]
                                )

In [ ]:
sample_net_power = sample_microgrid.get_net_power(
    df_hourlyload_1year=df_load_hourly_2017,
    df_weather_hourly_1year=df_weather_hourly_2017,
    plot=False
)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sample_net_power.index, y=sample_net_power['Net Power (kW)'], name='Net Power'))
fig.add_trace(go.Scatter(x=sample_net_power.index, y=sample_net_power['Net Power Storage (kW)'], name='Net Power with Storage', line=dict(width=2, dash='dash')))
# Add load line
fig.add_trace(go.Scatter(x=sample_net_power.index, y=sample_net_power['Load (kW)'], name='Load', line=dict(color='green', width=1)))
# Add power generation lines
fig.add_trace(go.Scatter(x=sample_net_power.index, y=sample_net_power['Power Output (kW)'], name='Power Output', line=dict(color='orange', width=1)))
fig.update_layout(title='Net Power Output Over Time (2017)',
                  xaxis_title='Timestamp',
                  yaxis_title='Power Output (kW)')
fig.show()

In [ ]:
# Get sample microgrid generator fuel consumtion
dg = DieselGenerator(diesel_price=1.8, fuel_consumption=0.37, price_per_rated_kw=375)
print("Storage diesel fuel consumption", dg.get_fuel_consumption(sample_net_power['Net Power Storage (kW)']))
# Get sum of net power storage over the year
print("Net power total balance: ", sample_net_power['Net Power Storage (kW)'].sum())
# Get som of net power over the year
print("Net power total balance: ", sample_net_power['Net Power (kW)'].sum())

In [ ]:
def get_micro_grid_cost_function(solar_price_psqm, 
                                    storage_price_pkw, 
                                    generator_price_per_rated_kw, 
                                    diesel_price, # this kinda serves as penalty for unreliability
                                    gen_fuel_consumption, 
                                    hi_power_turbine_price, 
                                    mid_power_turbine_price, 
                                    low_power_turbine_price, 
                                    list_of_load_dfs,
                                    list_of_weather_dfs,
                                    wf_builder: WindFarmBuilder,
                                    operation_time
                                    ):

    # Define wind 
    gazelle_turbine = WindTurbine(blade_radius=5, # [m]
                                performance_coefficient=0.4, # [-]
                                air_density=1.225, # [kg/m^3]
                                nominal_power=20, # [kW] 
                                nominal_wind_speed=13, # [m/s]
                                cut_out_speed=20, # [m/s] 
                                cut_in_speed=4, # [m/s] 
                                price=hi_power_turbine_price, # EUR from alibaba
                                lifetime=22.5, # years 
                                height=18) # [m]

    aircon_hawt_turbine = WindTurbine(blade_radius=3.5, # [m]
                                performance_coefficient=0.35, # [-]
                                air_density=1.225, # [kg/m^3]
                                nominal_power=10, # [kW] 
                                nominal_wind_speed=11, # [m/s]
                                cut_out_speed=32, # [m/s] 
                                cut_in_speed=2.5, # [m/s] 
                                price=mid_power_turbine_price, # EUR 
                                lifetime=25, # years 
                                height=18) # [m]

    travere_turbine = WindTurbine(blade_radius=1.6, # [m]
                                performance_coefficient=0.35, # [-]
                                    air_density=1.225, # [kg/m^3]
                                    nominal_power=1.6, # [kW]
                                    nominal_wind_speed=10, # [m/s]
                                    cut_out_speed=60, # [m/s]
                                    cut_in_speed=2.5 , # [m/s]
                                    price=low_power_turbine_price, # EUR
                                    lifetime=25, # years
                                    height=12) # [m]

    def cost_function(x):
        """
        x[0] is number of hi power turbines
        x[1] is number of mid power turbines
        x[2] is number of low power turbines
        x[3] is area of solar panels sqm
        x[4] is storage capacity in kWh
        """

        num_hi_power_turbines = int(round(x[0]))
        num_mid_power_turbines = int(round(x[1]))
        num_low_power_turbines = int(round(x[2]))
        # Build MicroGrid
        sample_wind_farm = wf_builder.build_wind_farm([gazelle_turbine, aircon_hawt_turbine, travere_turbine], [num_hi_power_turbines, num_mid_power_turbines, num_low_power_turbines])
        sample_solar_farm = SolarFarm(area=x[3], # [m^2] 
                                        pricepersqm= solar_price_psqm # in E
                                        )
        sample_microgrid = MicroGrid(windfarm = sample_wind_farm,
                                        solarfarm= sample_solar_farm,
                                        storage_capacity_kwh= x[4], # [kWh]
                                        storage_efficiency=0.8, # [-] 
                                        storage_price_per_kw= storage_price_pkw # [EUR]
                                        )
        diesel_gen = DieselGenerator(price_per_rated_kw=generator_price_per_rated_kw, # [EUR]
                                    diesel_price=diesel_price, # [EUR]
                                    fuel_consumption=gen_fuel_consumption # [L/kWh]
                                    )
        generator_required_powers = []
        generator_running_costs = []

        for load_df, weather_df in zip(list_of_load_dfs, list_of_weather_dfs):
            net_power = sample_microgrid.get_net_power(df_hourlyload_1year=load_df,
                                                        df_weather_hourly_1year=weather_df,
                                                        plot=False)
            generator_required_powers.append(diesel_gen.get_generator_specs(net_power['Net Power Storage (kW)']))

            generator_running_costs.append(diesel_gen.get_running_cost(net_power['Net Power Storage (kW)']))
        
        # Calculate total cost = microgrid cost + install cost of diesel generator + fuel cost of diesel generator
        microgrid_cost = sample_microgrid.get_total_price()
        diesel_gen_install_cost = np.max(generator_required_powers) * generator_price_per_rated_kw
        mean_fuel_cost = np.mean(generator_running_costs)
        total_cost = microgrid_cost + diesel_gen_install_cost + mean_fuel_cost * operation_time
        
        return total_cost
    return cost_function

In [ ]:
default_solar_price_psqm = 325
default_storage_price_pkw = 1000
default_generator_price_per_rated_kw = 375
default_diesel_price = 1.8
default_gen_fuel_consumption = 0.37
default_hi_power_turbine_price = 60000
default_mid_power_turbine_price = 35000
default_low_power_turbine_price = 7000


realistic_scenario = get_micro_grid_cost_function(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25 # years
                                                )

expensive_diesel_scenario = get_micro_grid_cost_function(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=2.3,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25 # years
                                                )

cheap_solar_scenario = get_micro_grid_cost_function(solar_price_psqm=200,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25 # years
                                                )

cheap_wind_scenario = get_micro_grid_cost_function(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=48000,
                                                mid_power_turbine_price=35000,
                                                low_power_turbine_price=5600,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25 # years
                                                )

cheap_batteries_scenario = get_micro_grid_cost_function(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=800,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25 # years
                                                )

super_expensive_diesel_scenario = get_micro_grid_cost_function(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=3,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25 # years
                                                )

super_cheap_storage_scenario = get_micro_grid_cost_function(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=666,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25 # years
                                                )


# Example usage
total_cost = realistic_scenario([7, 0, 0, 250, 0])
print("Total cost of installing the microgrid and running it for 25 years is: ", total_cost, " EUR")

# Set bounds for the variables
# x[0] is number of hi power turbines
# x[1] is number of mid power turbines
# x[2] is number of low power turbines
# x[3] is area of solar panels sqm
# x[4] is storage capacity in kWh

# Define bounds for the variables
bounds = [(0, 50),  # Number of high power turbines
          (0, 100),  # Number of mid power turbines
          (0, 100),  # Number of low power turbines
          (0, 25000),  # Area of solar panels in sqm
          (0, 15000)]  # Storage capacity in kWh

In [ ]:
def get_optimized_results(cost_function, bounds, name: str, maxiter = 200, export = True):
    # Check if file already exists, if it does return the results from it
    try:
        with open(f'./results/{name}_{maxiter}.json') as json_file:
            result_dict = json.load(json_file)
            print(f"{name} already exists.")
            return result_dict
    except:
        pass
    # Run differential evolution
    # x[0] is number of hi power turbines
    # x[1] is number of mid power turbines
    # x[2] is number of low power turbines
    # x[3] is area of solar panels sqm
    # x[4] is storage capacity in kWh
    result = differential_evolution(cost_function, bounds, maxiter=maxiter, disp=True)
    # Create dictionary of results
    result_dict = {'Number of hi power turbines': int(round(result.x[0])),
                   'Number of mid power turbines': int(round(result.x[1])),
                   'Number of low power turbines': int(round(result.x[2])),
                   'Area of solar panels (sqm)': result.x[3],
                   'Storage capacity (kWh)': result.x[4],
                   'Total cost (EUR)': result.fun}
    
    if export:
        with open(f'./results/{name}_{maxiter}.json', 'w') as fp:
            json.dump(result_dict, fp)
    return result_dict

In [ ]:
get_optimized_results(realistic_scenario, bounds, 'realistic_result', maxiter=10, export=True)
get_optimized_results(expensive_diesel_scenario, bounds, 'expensive_diesel_result', maxiter=10, export=True)
get_optimized_results(cheap_solar_scenario, bounds, 'cheap_solar_result', maxiter=10, export=True)
get_optimized_results(cheap_wind_scenario, bounds, 'cheap_wind_result', maxiter=10, export=True)
get_optimized_results(cheap_batteries_scenario, bounds, 'cheap_batteries_result', maxiter=10, export=True)
get_optimized_results(super_expensive_diesel_scenario, bounds, 'super_expensive_diesel_result', maxiter=10, export=True)
get_optimized_results(super_cheap_storage_scenario, bounds, 'super_cheap_storage_result', maxiter=10, export=True)

realistic_result = get_optimized_results(realistic_scenario, bounds, 'realistic_result', maxiter=50, export=True)
expensive_diesel_result = get_optimized_results(expensive_diesel_scenario, bounds, 'expensive_diesel_result', maxiter=50, export=True)
cheap_solar_result = get_optimized_results(cheap_solar_scenario, bounds, 'cheap_solar_result', maxiter=50, export=True)
cheap_wind_result = get_optimized_results(cheap_wind_scenario, bounds, 'cheap_wind_result', maxiter=50, export=True)
cheap_batteries_result = get_optimized_results(cheap_batteries_scenario, bounds, 'cheap_batteries_result', maxiter=50, export=True)
super_expensive_diesel_result = get_optimized_results(super_expensive_diesel_scenario, bounds, 'super_expensive_diesel_result', maxiter=50, export=True)
super_cheap_storage_result = get_optimized_results(super_cheap_storage_scenario, bounds, 'super_cheap_storage_result', maxiter=50, export=True)

In [ ]:
# Basically re run the simulation with the optimized results to get generator fuel consumed, generator running cost, generator install cost
def get_generator_results(solar_price_psqm, 
                                storage_price_pkw, 
                                generator_price_per_rated_kw, 
                                diesel_price, # this kinda serves as penalty for unreliability
                                gen_fuel_consumption, 
                                hi_power_turbine_price, 
                                mid_power_turbine_price, 
                                low_power_turbine_price, 
                                list_of_load_dfs,
                                list_of_weather_dfs,
                                wf_builder: WindFarmBuilder,
                                operation_time,
                                optimized_results: dict
                                ):

    # Define wind 
    gazelle_turbine = WindTurbine(blade_radius=5, # [m]
                            performance_coefficient=0.4, # [-]
                            air_density=1.225, # [kg/m^3]
                            nominal_power=20, # [kW] 
                            nominal_wind_speed=13, # [m/s]
                            cut_out_speed=20, # [m/s] 
                            cut_in_speed=4, # [m/s] 
                            price=hi_power_turbine_price, # EUR from alibaba
                            lifetime=22.5, # years 
                            height=18) # [m]

    aircon_hawt_turbine = WindTurbine(blade_radius=3.5, # [m]
                            performance_coefficient=0.35, # [-]
                            air_density=1.225, # [kg/m^3]
                            nominal_power=10, # [kW] 
                            nominal_wind_speed=11, # [m/s]
                            cut_out_speed=32, # [m/s] 
                            cut_in_speed=2.5, # [m/s] 
                            price=mid_power_turbine_price, # EUR 
                            lifetime=25, # years 
                            height=18) # [m]

    travere_turbine = WindTurbine(blade_radius=1.6, # [m]
                                performance_coefficient=0.35, # [-]
                                air_density=1.225, # [kg/m^3]
                                nominal_power=1.6, # [kW]
                                nominal_wind_speed=10, # [m/s]
                                cut_out_speed=60, # [m/s]
                                cut_in_speed=2.5 , # [m/s]
                                price=low_power_turbine_price, # EUR
                                lifetime=25, # years
                                height=12) # [m]
    x = [optimized_results['Number of hi power turbines'], optimized_results['Number of mid power turbines'], optimized_results['Number of low power turbines'], optimized_results['Area of solar panels (sqm)'], optimized_results['Storage capacity (kWh)']]
    def gen_specs(x):
        """
        x[0] is number of hi power turbines
        x[1] is number of mid power turbines
        x[2] is number of low power turbines
        x[3] is area of solar panels sqm
        x[4] is storage capacity in kWh
        """

        num_hi_power_turbines = int(round(x[0]))
        num_mid_power_turbines = int(round(x[1]))
        num_low_power_turbines = int(round(x[2]))
        # Build MicroGrid
        sample_wind_farm = wf_builder.build_wind_farm([gazelle_turbine, aircon_hawt_turbine, travere_turbine], [num_hi_power_turbines, num_mid_power_turbines, num_low_power_turbines])
        sample_solar_farm = SolarFarm(area=x[3], # [m^2] 
                                        pricepersqm= solar_price_psqm # in E
                                        )
        sample_microgrid = MicroGrid(windfarm = sample_wind_farm,
                                        solarfarm= sample_solar_farm,
                                        storage_capacity_kwh= x[4], # [kWh]
                                        storage_efficiency=0.8, # [-] 
                                        storage_price_per_kw= storage_price_pkw # [EUR]
                                        )
        diesel_gen = DieselGenerator(price_per_rated_kw=generator_price_per_rated_kw, # [EUR]
                                    diesel_price=diesel_price, # [EUR]
                                    fuel_consumption=gen_fuel_consumption # [L/kWh]
                                    )
        generator_required_powers = []
        generator_running_costs = []

        for load_df, weather_df in zip(list_of_load_dfs, list_of_weather_dfs):
            net_power = sample_microgrid.get_net_power(df_hourlyload_1year=load_df,
                                                        df_weather_hourly_1year=weather_df,
                                                        plot=False)
            generator_required_powers.append(diesel_gen.get_generator_specs(net_power))

            generator_running_costs.append(diesel_gen.get_running_cost(net_power))
        
        # Calculate total cost = microgrid cost + install cost of diesel generator + fuel cost of diesel generator
        microgrid_cost = sample_microgrid.get_total_price()
        diesel_gen_install_cost = np.max(generator_required_powers) * generator_price_per_rated_kw
        # mean fuel cost if mean yearly cost of fuel
        mean_fuel_cost = np.mean(generator_running_costs)
        total_cost = microgrid_cost + diesel_gen_install_cost + mean_fuel_cost * operation_time
        
        return {
            'Generator kw rating': np.max(generator_required_powers),
            'Generator install cost': diesel_gen_install_cost,
            'Generator mean fuel cost': mean_fuel_cost,
            'Generator mean annual fuel consumption in liters': mean_fuel_cost / diesel_price,
        }
    return gen_specs(x)

In [ ]:
realistic_gen_specs = get_generator_results(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25, # years
                                                optimized_results=realistic_result
                                                )

expensive_diesel_gen_specs = get_generator_results(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=2.3,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25, # years
                                                optimized_results=expensive_diesel_result
                                                )

cheap_solar_gen_specs = get_generator_results(solar_price_psqm=200,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25, # years
                                                optimized_results=cheap_solar_result
                                                )

cheap_wind_gen_specs = get_generator_results(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=48000,
                                                mid_power_turbine_price=35000,
                                                low_power_turbine_price=5600,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25, # years
                                                optimized_results=cheap_wind_result
                                                )

cheap_batteries_gen_specs = get_generator_results(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=800,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25, # years
                                                optimized_results=cheap_batteries_result
                                                )

super_expensive_diesel_gen_specs = get_generator_results(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=default_storage_price_pkw,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=3,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25, # years
                                                optimized_results=super_expensive_diesel_result
                                                )

super_cheap_storage_gen_specs = get_generator_results(solar_price_psqm=default_solar_price_psqm,
                                                storage_price_pkw=666,
                                                generator_price_per_rated_kw=default_generator_price_per_rated_kw,
                                                diesel_price=default_diesel_price,
                                                gen_fuel_consumption=default_gen_fuel_consumption,
                                                hi_power_turbine_price=default_hi_power_turbine_price,
                                                mid_power_turbine_price=default_mid_power_turbine_price,
                                                low_power_turbine_price=default_low_power_turbine_price,
                                                list_of_load_dfs=[df_load_hourly_2013, df_load_hourly_2014, df_load_hourly_2015, df_load_hourly_2017, df_load_hourly_2018],
                                                list_of_weather_dfs=[df_weather_hourly_2013, df_weather_hourly_2014, df_weather_hourly_2015, df_weather_hourly_2017, df_weather_hourly_2018],
                                                wf_builder=wf_builder,
                                                operation_time=25, # years
                                                optimized_results=super_cheap_storage_result
                                                )

In [ ]:
# Print results
print("Realistic scenario results", realistic_result)
print("Expensive diesel scenario results", expensive_diesel_result)
print("Cheap solar scenario results", cheap_solar_result)
print("Cheap wind scenario results", cheap_wind_result)
print("Cheap batteries scenario results", cheap_batteries_result)
print("Super expensive diesel scenario results", super_expensive_diesel_result)
print("Super cheap storage scenario results", super_cheap_storage_result)

In [ ]:
# Print gen specs
print("Realistic gen specs: ", realistic_gen_specs)
print("Expensive diesel gen specs: ", expensive_diesel_gen_specs)
print("Cheap solar gen specs: ", cheap_solar_gen_specs)
print("Cheap wind gen specs: ", cheap_wind_gen_specs)
print("Cheap batteries gen specs: ", cheap_batteries_gen_specs)
print("Super expensive diesel gen specs: ", super_expensive_diesel_gen_specs)
print("Super cheap storage gen specs: ", super_cheap_storage_gen_specs)

In [ ]:
realistic_total_cost = realistic_scenario([7, 0, 0, 353, 0, 0])
print("Realistic total cost: ", realistic_total_cost)